In [ ]:
from fastai.vision.all import *
import pandas as pd
set_seed(3)

In [ ]:
trn_path = '/kaggle/input/zindidata/images/images'
files    = get_image_files(trn_path) 

In [ ]:
Train            = pd.read_csv('/kaggle/input/zindidata/Train.csv')
Test             = pd.read_csv('/kaggle/input/zindidata/Test.csv')
SampleSubmission = pd.read_csv('/kaggle/input/zindidata/SampleSubmission.csv')

In [ ]:
damage = Train.damage.unique()

In [ ]:
Train.head(2)

In [ ]:
def train(arch, item, batch, epoch=7):
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = '/kaggle/input/zindidata/images/images/'),
                       get_y = ColReader('damage'),
                       splitter = RandomSplitter(valid_pct = 0.2, seed = 3),
                       item_tfms = item, #Resize(320, method = 'squish'),
                       batch_tfms = batch, #aug_transforms(size = 128, min_scale = 0.75)
                      )

    dls = dblock.dataloaders(Train)
    learn = vision_learner(dls, arch, loss_func = nn.CrossEntropyLoss(), metrics=accuracy).to_fp16()
    learn.fine_tune(epoch, 0.018)    
    return learn

In [ ]:
torch.cuda.empty_cache()
import torch

# Print GPU memory summary
print(torch.cuda.memory_summary())


In [ ]:
learn1 = train('convit_small.fb_in1k', Resize(256, method='squish')
               , aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=13.2, min_zoom=0.6,
                 max_zoom=1.9, max_lighting=0.4,
                 max_warp=0.4, p_affine=0.60, size = 224, min_scale= 0.7)
               , epoch=10)

In [ ]:
learn2 = train('convnext_small.in12k', Resize(256, method='squish'), aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=13.2, min_zoom=0.6,
                 max_zoom=1.9, max_lighting=0.4,
                 max_warp=0.4, p_affine=0.60, size = 224, min_scale= 0.7), epoch=10)

In [ ]:
learn3 = train('deit_small_distilled_patch16_224.fb_in1k', Resize(256, method='squish'), aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=13.2, min_zoom=0.6,
                 max_zoom=1.9, max_lighting=0.4,
                 max_warp=0.4, p_affine=0.52, size = 224, min_scale= 0.7), epoch=10)

In [ ]:
valid = learn1.dls.valid
preds1 , targets = learn1.tta(dl=valid)
accuracy(preds1 , targets)

In [ ]:
preds1 = F.softmax(preds1, dim=1)
preds1

In [ ]:
valid = learn2.dls.valid
preds2 , targets = learn2.tta(dl=valid)
accuracy(preds2 , targets)

In [ ]:
preds2 = F.softmax(preds2, dim=1)
preds2

In [ ]:
valid = learn3.dls.valid
preds3 , targets = learn3.tta(dl=valid)
accuracy(preds3 , targets)

In [ ]:
preds3 = F.softmax(preds3, dim=1)
preds3

In [ ]:
ensemble_preds = (preds1 + preds2 + preds3) / 3
accuracy(ensemble_preds , targets)

In [ ]:
ensemble_preds

In [ ]:
test_dl = learn1.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds1 , _ = learn1.tta(dl=test_dl)

In [ ]:
preds1 = F.softmax(preds1, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds1[:, 0].squeeze().numpy(),
    'G' : preds1[:, 1].squeeze().numpy(),
    'ND': preds1[:, 2].squeeze().numpy(),
    'WD': preds1[:, 3].squeeze().numpy(),
    'other': preds1[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convit2.csv', index=False)

In [ ]:
submission_sf

In [ ]:
test_dl = learn2.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds2 , _ = learn2.tta(dl=test_dl)

In [ ]:
preds2 = F.softmax(preds2, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds2[:, 0].squeeze().numpy(),
    'G' : preds2[:, 1].squeeze().numpy(),
    'ND': preds2[:, 2].squeeze().numpy(),
    'WD': preds2[:, 3].squeeze().numpy(),
    'other': preds2[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convnext2.csv', index=False)

In [ ]:
submission_sf

In [ ]:
test_dl = learn3.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds3 , _ = learn3.tta(dl=test_dl)

In [ ]:
preds3 = F.softmax(preds3, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds3[:, 0].squeeze().numpy(),
    'G' : preds3[:, 1].squeeze().numpy(),
    'ND': preds3[:, 2].squeeze().numpy(),
    'WD': preds3[:, 3].squeeze().numpy(),
    'other': preds3[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('deit_small2.csv', index=False)

In [ ]:
submission_sf

In [ ]:
ens_preds = (preds1 + preds2 + preds3) / 3

In [ ]:
submission_sf = pd.DataFrame({
    'ID': Test['ID'],
    'DR': ens_preds[:, 0].squeeze().numpy(),
    'G' : ens_preds[:, 1].squeeze().numpy(),
    'ND': ens_preds[:, 2].squeeze().numpy(),
    'WD': ens_preds[:, 3].squeeze().numpy(),
    'other': ens_preds[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('ens2.csv', index=False)

In [ ]:
submission_sf